In [ ]:
import numpy as np

In [ ]:
# test a unconventional contract

a = np.arange(25).reshape(5,5)
b = np.arange(25).reshape(5,5)

# print(a)
# print(b)

c = np.einsum("ik,il->ikl", a, b)

# print(c)
# print(c.shape)

c_bench = np.zeros((5,5,5))

for i in range(5):
    for j in range(5):
        for k in range(5):
            c_bench[i,j,k] = a[i,j] * b[i,k]

# print(c_bench)

print(np.allclose(c, c_bench))

In [ ]:
# test einsum_path

N = 12

ao_2_mo = np.random.rand(N, N)
eri_ao = np.random.rand(N, N, N, N)

# timer 

import time

start1 = time.time()
eri_mo = np.einsum("pqrs,pi,qj,rk,sl->ijkl", eri_ao, ao_2_mo, ao_2_mo, ao_2_mo, ao_2_mo)
end1 = time.time()

print("time for einsum: ", end1 - start1)

# test einsum_path

start2 = time.time()
path = np.einsum_path("pqrs,pi,qj,rk,sl->ijkl", eri_ao, ao_2_mo, ao_2_mo, ao_2_mo, ao_2_mo, optimize="greedy")

print(path[0])
print(path[1])
path = path[0]

_ = np.einsum("pqrs,pi,qj,rk,sl->ijkl", eri_ao, ao_2_mo, ao_2_mo, ao_2_mo, ao_2_mo, optimize=path)
end2 = time.time()

print("time for einsum_opt: ", end2 - start2)

print("speedup: ", (end1 - start1) / (end2 - start2))

In [ ]:
# a model for ISDF with benchmark 

N = 3
M = 6

M_mat = np.random.rand(M, M)
mu_L = np.random.rand(N, M)

eri_ao = np.einsum("rs,ir,jr,ks,ls->ijkl", M_mat, mu_L, mu_L, mu_L, mu_L)

# benchmark

eri_ao_bench = np.zeros((N, N, N, N))

for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                for r in range(M):
                    for s in range(M):
                        eri_ao_bench[i,j,k,l] += M_mat[r,s] * mu_L[i,r] * mu_L[j,r] * mu_L[k,s] * mu_L[l,s]

print(np.allclose(eri_ao, eri_ao_bench))

In [ ]:
# a model for ISDF 

N = 16 
M = 64 

M_mat = np.random.rand(M, M)
mu_L = np.random.rand(N, M)


start1 = time.time()
eri_ao = np.einsum("rs,ir,jr,ks,ls->ijkl", M_mat, mu_L, mu_L, mu_L, mu_L)
end1 = time.time()

print("time for einsum: ", end1 - start1)

# test einsum_path

start2 = time.time()
path = np.einsum_path("rs,ir,jr,ks,ls->ijkl", M_mat, mu_L, mu_L, mu_L, mu_L, optimize="greedy")

print(path[0])
print(path[1])

path = path[0]

eri_ao2 = np.einsum("rs,ir,jr,ks,ls->ijkl", M_mat, mu_L, mu_L, mu_L, mu_L, optimize=path)

end2 = time.time()

print("time for einsum_opt: ", end2 - start2)

print("speedup: ", (end1 - start1) / (end2 - start2))

print(np.allclose(eri_ao, eri_ao2))